In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# specify imports to be used in the notebook
import tensorflow as tf
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import numpy as np
import PIL
import pathlib
import cv2

from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
CATEGORIES = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']

IMG_SIZE=255

DATADIR  = '../input/plant-seedlings-classification/train'
DATADIR_TEST  = '../input/plant-seedlings-classification/test'

In [ ]:
TRAIN_DATA_DIR = DATADIR
batch_size = 16
# create data generators
def data_generators():

    # apply random transformations on training data
    train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=180,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split = 0.1,
    )

    # define training data generator
    train_gen = train_data_generator.flow_from_directory(
        directory = TRAIN_DATA_DIR,
        target_size = (IMG_SIZE, IMG_SIZE),
        color_mode = 'rgb',
        class_mode = "categorical",
        batch_size = batch_size,
        subset = 'training',
    )

    # define validation data generator
    validation_gen = train_data_generator.flow_from_directory(
        directory = TRAIN_DATA_DIR,
        color_mode = 'rgb',
        class_mode = "categorical",
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        subset = 'validation',
    )

    return train_gen, validation_gen

train_gen, validation_gen = data_generators()

In [ ]:
training_data=[]

def create_training_data():
    for category in CATEGORIES:

        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                
                # Pre-processing (via function pre_process) ##
                
#                 image = pre_process(new_array)
                image = new_array
                training_data.append([image,class_num])
            except Exception as e:
                print(e)
                pass
            
create_training_data()  

In [ ]:
import numpy as np

X_train=[]
y_train=[]

for categories, label in training_data:
    X_train.append(categories)
    y_train.append(label)
# X = np.array(X).reshape(train_len,-1)

y_train = np.array(y_train)

y_train.shape

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, num_classes=len(CATEGORIES))
# y_test = np_utils.to_categorical(y_test, num_classes=len(CATEGORIES))

X_train = np.array(X_train).astype('float32')

In [ ]:
print(X_train.shape)

print(y_train.shape)

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

X_train, y_train = unison_shuffled_copies(X_train, y_train)

In [ ]:
def create_model(base_model):
    base_model.trainable = True
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    prediction_layer = tf.keras.layers.Dense(12, activation='softmax')(global_average_layer)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

# define appropriate callbacks
def training_callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'efficientnet_best.h5',
        monitor = 'val_accuracy', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_accuracy', min_delta = 1e-2, patience = 10, verbose = 1,
        mode = 'max', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

def model2():
    width, height = IMG_SIZE,IMG_SIZE
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    random_flip_model = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(input_model)
    random_rotation_model = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(random_flip_model)
    random_zoom_model = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(random_rotation_model)
    
    main_model = tf.keras.applications.EfficientNetB5(include_top = False, weights = 'imagenet')(random_zoom_model)
    main_model.trainable = True
    flattened_model = tf.keras.layers.Flatten()(main_model)
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(0.0005)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model

In [ ]:
model2 = model2()

In [ ]:
batch_size = 16
epochs = 100

history = model2.fit(X_train, y_train,
                    batch_size=batch_size,
                    steps_per_epoch=len(X_train)//batch_size, 
                    epochs=epochs, 
                    callbacks = training_callbacks(),
                    validation_split=0.1)
    
# history = model2.fit(train_gen,
#           callbacks  = training_callbacks(),
#           epochs = epochs,
#           steps_per_epoch=train_gen.samples // batch_size,
#           validation_data = validation_gen,
#           validation_steps = validation_gen.samples // batch_size,)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model2.save('efficientnet.h5')

In [ ]:
from IPython.display import FileLink
FileLink(r'efficientnet.h5')

In [ ]:
from IPython.display import FileLink
FileLink(r'efficientnet_best.h5')

In [ ]:
testing_data =[]
def create_test_data():

    for img in os.listdir(DATADIR_TEST):
        try:
            img_array=cv2.imread(os.path.join(DATADIR_TEST,img))
            new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))

            ## Pre-processing (via function pre_process)

#             image = pre_process(new_array)
            image = new_array
            testing_data.append([image,img])
        except Exception as e:
            print(e)
            pass
create_test_data()

In [ ]:
import numpy as np

X_test=[]
y_test=[]

for categories, label in testing_data:
    X_test.append(categories)
    y_test.append(label)
# X = np.array(X).reshape(train_len,-1)

y_test = np.array(y_test)

y_test.shape

In [ ]:
X_test = np.array(X_test).astype('float32')

In [ ]:
model = tf.keras.models.load_model('efficientnet.h5')

In [ ]:
model_preds = model.predict(X_test)
classes = []

for data in range(0, model_preds.shape[0]):
    pred_index = model_preds[data, :].argmax(axis = -1)
    classes += [CATEGORIES[pred_index]]


In [ ]:
# generate submission csv
output_predictions = pd.DataFrame()
output_predictions['file'] = y_test
output_predictions['file'] = output_predictions['file'].str.replace(r'test/', '')
output_predictions['species'] = classes
output_predictions.to_csv('output_efficientnet.csv', index = False)